In [18]:
# RLest_20231215ver
# Cross Entropyで推定したる！！
import pandas as pd
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random

In [19]:
df_data = pd.read_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec.csv')
search_folder = "/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/split_files/group_3_1/group_2"
file_list = list(os.listdir(search_folder))

df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_link_post.csv')
L = len(df_link)

df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_node.csv')
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')

In [23]:

#######################################
####### パラメタ&タイムステップ設定 #######
#######################################

xdim = 2
x0 = np.zeros(xdim)
x0[0] = -1
x0[-1] = 0.8

sigma1 = 2.0 # BLEの減衰定数．推定したいのと，混雑に影響されそう
sigma2 = 15 # 是非とも推定
T = 11 # 20ステップ以内ならT=21
#b = np.array([2, 15]) ## b[0]がBLE減衰定数，b[1]が正規分布の誤差分散

ddata = [66, 67, 68] # 吸収リンク linkidで指定
D = len(ddata)


In [24]:
#######################
######### 準備 ########
#######################

###### 各linkのoとdの座標を入れておく配列 ###### 
link_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0]
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)
    link_loc_array.append(loc_tuple) 


###### 各ビーコンの座標を入れておく配列 ###### 
beacon_loc_array = []
for j in range(len(df_ble)):
    beacon_id = df_ble.loc[j, 'ID']
    x_j = df_ble[df_ble['ID'] == beacon_id]['x'].iloc[0]
    y_j = df_ble[df_ble['ID'] == beacon_id]['y'].iloc[0]
    z_j = df_ble[df_ble['ID'] == beacon_id]['floor'].iloc[0]
    
    beacon_loc = np.array([x_j, y_j, z_j])
    beacon_loc_array.append(beacon_loc) 


###### ビーコンとリンク線分の中点間距離を返す関数 ######
def dist_to_mid(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)

    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算
    distance = np.linalg.norm(x - mid_point)
    
    return distance


###### d_arrayの用意 ###### # 各リンクの中点とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))
for i in range(len(df_link)):
    p_o = link_loc_array[i][0] # o座標 # i=1の時
    p_d = link_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = dist_to_mid(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離


###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
# 15分間のデータでやるならtimstep18secなら50step, 20secなら45step
dens_jt = np.zeros((23, 50), dtype = int)
max_jt = np.zeros((23, 50), dtype = int)
mean_jt = np.zeros((23, 50), dtype = int)
#count_jt = np.zeros((23, 45), dtype = int)

grouped = df_data.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count


########################################
####### リンク接続行列＆プリズム準備 #######
########################################

def Imaking(link_data): 
    n = len(link_data)
    I = np.eye(n) # 全リンクに滞在リンク付与
    
    for i in range(n):

        if (i < 25) or (i >=  65): # 階段リンク以外の場合→これまで通り
            O = link_data.loc[i, 'o'] 
            D = link_data.loc[i, 'd'] 
            for j in range(n):
                if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                    I[i, j] = 1

        if 25 <= i < 65: ### iが25~64の場合→階段リンクの場合
            O = link_data.loc[i, 'o'] # onode
            D = link_data.loc[i, 'd'] # dnode

            for j in range(n):
                if link_data.loc[j, 'o'] == D: # DをOnodeとするリンクを1にしている
                    I[i, j] = 1
                
                if link_data.loc[j, 'o'] == D or link_data.loc[j, 'd'] == D: # odいずれかがDと一致するリンク[からの]遷移は0
                    I[j, i] = 0
                
                if link_data.loc[j, 'o'] == O or link_data.loc[j, 'd'] == O: # odいずれかがOと一致するリンクへの遷移は0
                    I[i, j] = 0

    return(I)

I = Imaking(df_link)

for i in range(26, 65, 2): # 階段には滞在リンク入れない
    I[i-1, i-1] = 0
    I[i, i] = 0

for i in range(25, 46): # 階段リンクに入ったら半分のところで折り返さない
    I[i, i+20] = 0
    I[i+20, i] = 0

for i in range(66, 69): # 吸収リンクからは吸収リンクにしか接続しない
    I[i-1, :] = 0 
    I[i-1, i-1] = 1 

# 起点Oからの到達可能性指示変数io
Itlist = np.zeros((T, L, L)) ## これは各状態に対して与えるので遷移回数T-1よりひとつ多くてT成分考える（timestep数がTなら状態数はT）
II = np.copy(I)
Itlist[0, :, :] = np.eye(L) 

for ts in range(1, T): 
    Itlist[ts, :, :] = II 
    II = np.dot(II, I)
    II = np.where(II > 0, 1, 0) 

# 終点Dからの到達可能性指示変数id ## これもIlist同様の考え方
Ittlist = np.zeros((T, L, L))
for ts in range(T): 
    Ittlist[ts, :, :] = np.transpose(Itlist[T - ts - 1, :, :]) 



/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_3219/1926983877.py:49: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算


In [25]:
#####################
#### od_list作成 #### # ビーコン列からODを特定（ちょっとよくないけどとりあえず）
#####################

saikyo_key = [1, 2, 3, 4, 5, 6]
yamate_key = [7, 8, 9, 10, 11, 12, 13]
discriminate_key = [19, 22, 23]
abs_key = [16, 20, 21]

od_list = pd.DataFrame(columns=['userid', 'o', 'd', 'abs', 'Ti'])

count_file = 0
for file_name in file_list:

    dfi = pd.read_csv(os.path.join(search_folder, file_name))   # file_path = os.path.join(search_folder, file_name)
    userid = int(file_name.split('.')[0])                       # userid = int(base_name)

    Ti = dfi['timestep'].nunique()                              # 個人の滞在時間 Tiまでにlast_linkにくる   
    grouped = dfi.groupby('timestep')
    dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

    ### 各人の各時刻の最大スコアのbeacon列 ###
    max_beacon_list = [] 
    for t in range(Ti):
        dfit = dfi_list[t] # 時刻tの全ログデータ
        ts = int(dfit.loc[0, 'timestep'])

        captured_beacon = dfit.groupby('ID')
        captured_beacon_list = [group.reset_index(drop=True) for name, group in captured_beacon]
        tdict = {}

        for j in range(len(dfit)):
            jid = dfit.loc[j, 'ID'] # そのログデータのビーコン
            max_rssi = max_jt[jid-1, ts-1] # そのビーコンの時刻tsでのmax_rssi
            rssi = dfit.loc[j, 'RSSI'] # そのログの電波強度
            jscore = max_rssi / rssi # このログのスコア→これを辞書のjidのところに足すイメージ
            if jid not in tdict:
                tdict[jid] = 0
            tdict[jid] += jscore  

        jmax = max(tdict, key = tdict.get) 
        max_beacon_list.append(jmax) #### なんかここ空になってる，なんで→append使う時はa = a.append()ってしないでa.append()だけにするらしいわ
        # これはGPTに言われるまで知らなかった，，，

    ### Oの確定 ### 
    first_beacon = max_beacon_list[0]
    first_dist_list = d_array[:, first_beacon-1] 
    first_dist_dict = {index + 1: value for index, value in enumerate(first_dist_list)} # keyはlinkidになっている．max_beaconと各リンクの距離が入ってる

    if any(item in discriminate_key for item in max_beacon_list): # 埼京オンリーにする
        filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in saikyo_key}
        max_key = min(filtered_dict, key = filtered_dict.get) # これが初期リンクとなる．埼京ホームのうち，最もmax_beaconに近いリンク
    else:
        filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in yamate_key}
        max_key = min(filtered_dict, key = filtered_dict.get)

    a0 = int(max_key)       # linkid
        
    #### Dの確定 #### 
    last_beacon = max_beacon_list[-1]

    if last_beacon not in abs_key: 
        continue
        
    last_link = 0
    if last_beacon == 16: # node39
        last_link = 25 
        abs_link = 46
    elif last_beacon == 20: # node28
        last_link = 17
        abs_link = 47
    elif last_beacon == 21: # node29
        last_link = 18   
        abs_link = 48   
    else:
        continue

    ## ここまで条件を満たせばod_listにデータを追加
    od_list.loc[count_file, 'userid'] = userid
    od_list.loc[count_file, 'o'] = a0
    od_list.loc[count_file, 'd'] = last_link
    od_list.loc[count_file, 'abs'] = abs_link
    od_list.loc[count_file, 'Ti'] = Ti

    count_file += 1

OD = len(od_list)


##########################
###### 時空間プリズム ######
##########################

Ilist = []
for i in range(OD):   

    # OD確定読み込み
    ao = od_list.loc[i, 'o'] - 1  # index
    ad = od_list.loc[i, 'd'] - 1  # index
    aabs = od_list.loc[i, 'abs'] - 1
    userid = od_list.loc[i, 'userid']

    Id = np.zeros((T-1, L, L)) # Id[t]は時刻tの時の利用可能遷移k→aを示す．最後のt=T-1のとき不要なので要素数はt=0~T-2のT-1個

    for ts in range(T-1): # Tまでに吸収されるのでdはaabs．遷移回数はT-1なのでこれでOK
        if ts == 0:
            Id[ts, ao, :] = I[ao, :] 
            continue
        
        alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, aabs, :])[0] ## ts=1（つまり第二回目の状態）のとき3番目の状態を見ている→OK
        ## Itlistの長さはT．最後はT-2で，そのときT-2+1=T-1, 
        for a in alist:
            if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, aabs, a]: # always True
                klist = np.where(I[:, a] == 1)[0]

                for k in klist:
                    if len(np.where(Id[ts - 1, :, k] == 1)[0]) != 0:
                        Id[ts, k, a] = 1 

    Ilist.append(Id)


In [26]:


#######################
###### 観測モデル #######
#######################

def qjx(rssi, max_rssi, x_loc, j, b):               # bは観測誤差分散のパラメータ．x_loc（x, y, z)にいたときビーコンjでrssiの観測が得られる確率
    j_loc = beacon_loc_array[j-1]                   # id=jのビーコンの位置座標
    #s = max_rssi
    d_est = 10 ** ((max_rssi - rssi)/(10*b[0]))     # 減衰パラメタb[0] rssi0→-40にしてみる
    d = np.linalg.norm(x_loc - j_loc) 
    f = norm.pdf(d, d_est, b[1])                    # 誤差分散b[1]
    omega = (100 + rssi) / 10 

    return omega * f


def Mmodel(Pall, b): # 観測尤度を返す # xの代わりにPallを入れる．とにかくこれが本命！！！肝！！！！！！

    count = 0 # 観測結果に入ったサンプル数（後で出力する）
    
    SLlist = [] 
    TLlist = []

    for od in range(OD):

        # odに対応するuserのファイルを読み込む（od_listのuserid列が実データとod_listを紐づけている）
        userid = od_list.loc[od, 'userid']
        file_name = f'{userid}.csv'
        dfi = pd.read_csv(os.path.join(search_folder, file_name)) 
        grouped = dfi.groupby('timestep')
        dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

        Ti = od_list.loc[od, 'Ti']              # 個人odの観測限界．時刻Tiでlast_linkに到達し，時刻Ti+1で吸収される
        ao = od_list.loc[od, 'o'] - 1           # index
        ad = od_list.loc[od, 'd'] - 1           # index
        aabs = od_list.loc[od, 'abs'] - 1

        #### プリズム用意 #### odを与えたらプリズム制約．時刻Ti+1つまりTi回の遷移でaoからabsまで行く．もしくは時刻TiつまりTi-1回の遷移でaoからlastまで行く
        Idm = np.zeros((Ti, L, L)) ## 1成分目の値が遷移回数．Tiにするならabsまで行くことを仮定している．そのときItimlistは状態数＝時間数なのでTi+1で良い
        Itmlist = np.zeros((Ti+1, L, L))
        IIm = np.copy(I)
        Itmlist[0, :, :] = np.eye(L)

        for tt in range(1, Ti+1): 
            Itmlist[tt, :, :] = IIm 
            IIm = np.dot(IIm, I) 
            IIm = np.where(IIm > 0, 1, 0)

        Ittmlist = np.zeros((Ti+1, L, L))
        for tt in range(Ti+1): 
            Ittmlist[tt, :, :] = np.transpose(Itmlist[(Ti+1) - tt - 1, :, :])

        for tt in range(Ti):
            if tt == 0:
                Idm[tt, ao, :] = I[ao, :] 
                continue

            alist = np.where(Itmlist[tt + 1, ao, :] == Ittmlist[tt + 1, aabs, :])[0] # ここadじゃなくてaabsが適切では????
            for a in alist:
                if Itmlist[tt + 1, ao, a] == Ittmlist[tt + 1, aabs, a]: # ここadじゃなくてaabsが適切では????
                    klist = np.where(I[:, a] == 1)[0] 

                    for k in klist:
                        if len(np.where(Idm[tt - 1, :, k] == 1)[0]) != 0:
                            Idm[tt, k, a] = 1 ### 時刻tにおいて，a(t)→a(t+1)への遷移がOKなところが1になっている                            

        SLmatrix = np.zeros((T, L)) # status_likelihood 状態数は遷移数+1
        TLmatrix = np.zeros((T-1, L, L)) # trans likelihood
        pmmatrix = np.zeros((T-1, L, L))  # p(m|a)のmatrixのはず
        pmstmatrix = np.zeros((T-1, L, L)) # 同時発生尤度のマトリックス
        
        SLmatrix[0, ao] = 1 # t = 0ではaoに確定的に存在するとする ####### ←ここがまずよくない．

        
        for t in range(1, T): #### t=tでは，k=a(t-1)からa=a(t)への遷移を考える．TLmatrixは遷移数＝T-1個しかないのでindexは0 ~ t-2

            #######################################
            ######### 1. 遷移発生尤度を計算 ##########
            #######################################

            if t <= Ti - 1:

                dfit = dfi_list[t] 
                ts = int(dfit.loc[0, 'timestep'])

                slsum = 0
                nlist = np.argsort(SLmatrix[t-1])[::-1]  # tsにおける状態尤度が大きい順に並び替えたインデックスリスト
                ## t=1のときSL[0]を参照するがaoしかないのでそのままaoを見ることになる．t=2のときSL[1]を参照，tのときt-1を参照
                ## ループは1からまでにしてt-1における尤度が大きいリンクに対して??実装?
                ## t-1で尤度大きいリンクをkとしてtのリンクをaとして扱った方が実態に近い

                klist = [] # つまり時刻t-1で状態尤度が高い上位のリンクをklistに入れている

                for i in range(L):
                    slsum += SLmatrix[t-1, nlist[i]]
                    klist.append(nlist[i])
                    if slsum >= 0.9:
                        break
                
                klist3 = [i+1 for i in klist]
                
                #print(f'時刻{t-1}での状態発生尤度の高かったリンクは{klist3}') ##

                for k in klist: ## t-1での滞在尤度が高いリンクに対して走査

                    for a in range(L): ## tのリンクはこっち

                        if Idm[t-1, k, a] == 0: ### t-1時点でk→aが利用可能か否かを見る ###### 
                            TLmatrix[t-1, k, a] = 0 ### 多分これでいいのか？
                            continue
                        
                        a_loc = link_loc_array[a]
                        a_o = a_loc[0]
                        a_d = a_loc[1]

                        ## 位置については一旦保留→aの上に確率的に存在するという仮定が望ましいが一旦リンクの中点にいるとする
                        x_loc = (a_o + a_d) / 2

                        pm = 1

                        for j in range(len(dfit)): ### ここでようやく各観測ログにアプローチ
                            ### rssi小さい観測がpmの極小化を招いている説もある，積をとっているので．
                            ### でも相対評価で尤度最大化しようとしているならそこまで気にならないはずだが
                            j_id = dfit.loc[j, 'ID']
                            j_rssi = dfit.loc[j, 'RSSI']
                            j_max_rssi = max_jt[j_id - 1, ts - 1] 

                            pm *= qjx(j_rssi, j_max_rssi, x_loc, j_id, b) # 観測方程式
                        
                        pmmatrix[t-1, k, a] = pm # 時刻tの観測尤度はリンクaの観測確率ね→ただしindexの関係でt-1に入れる．つまりt=1にat-1=a0=k→at=a1=aの観測尤度がpm[0]に入ってる
                        #print(f'時刻{t}でリンクa(t)つまり{a+1}の観測尤度{pm}')
                
                # 遷移尤度のnume pmst 時刻tにおいてk(t-1)→a(t)への遷移が観測された確率がt-1に入ってる
                # 時刻tにおいてk(t-1)→a(t)への遷移確率がPall[t-1]に入っている=時刻t-1にk(t-1)→a(t)に遷移する確率がPall[t-1]ということ
                # つまり時刻tにおけるat→at+1への遷移確率はPall[t]に入っているということか．で，aT-1→aTへの遷移確率まで入っているがaT-2→aT-1までしか使わない
                pmstmatrix[t-1, :, :] = pmmatrix[t-1, :, :] * Pall[od, t-1, :, :] # 観測確率pm*遷移確率（正規化前の観測尤度）

                # 正規化
                deno = np.sum(pmstmatrix[t-1, :, :], axis=1) # np.sum(pmstmatrix[t-1], axis=1)になってた
                deno = (deno == 0) * 1 + (deno != 0) * deno ## 観測尤度の分母
                
                deno_rep = np.tile(deno, L).reshape(L, L).T ##### これがないと変になった

                pmstmatrix[t-1, :, :] = pmstmatrix[t-1, :, :] / deno_rep ## これが観測尤度

                sl = SLmatrix[t-1, :] # このままでは行ベクトル
                sl_col = sl[:, None] #### ここがRと文法異なる部分

                TLmatrix[t-1, :, :] = pmstmatrix[t-1, :, :] * sl_col # 同時発生尤度を計算してる（SLmatrixはindexが1増えてるので，同じt-1でもひとつ前の段階を指名している
                # 時刻tにk(a_t-1)からa(a_t)への遷移が発生する尤度

                # 非ゼロ要素のインデックスを取得
                non_zero_indices = np.transpose(np.nonzero(TLmatrix[t-1, :, :])) +1

                # タプル形式のリストに変換
                non_zero_tuples = [tuple(index) for index in non_zero_indices]

            elif t == Ti:
                TLmatrix[t-1, ad, aabs] = 1 ### それ以外はそのまま→0のまま
            
            else:
                TLmatrix[t-1, aabs, aabs] = 1 ### それ以外はそのまま→0のまま


            ########################################
            ########## 2. 状態発生尤度を定義 ##########
            ########################################
    
            if t <= Ti - 2: # 観測プリズム（ao, ad, Tiから決定）Idm内の状態しか取り得ない．k(a_t-1)からa(a_t)への遷移がOKのときIdmt[t-1]が1

                dfit = dfi_list[t] # t<len(dfit)=Tiにおいて指定可能
                ts = int(dfit.loc[0, 'timestep'])

                for i in range(L): ## iは遷移先リンク，つまりa(t)の候補ということになる

                    klist = np.where(Idm[t-1, :, i] == 1)[0] ## 前時刻参照．時刻tにおいてリンクi=a(t)に遷移可能な前リンクk=a(t-1)を探し，klistに入れている．この時，構造化NWと観測情報からリンク候補はかなり絞られている

                    if len(klist) == 0:
                        continue

                    kTL = 0
                    for k in klist: # このiに移動できる元リンク（t-1時点でのリンクa(t-1)）
                        kTL += TLmatrix[t-1, k, i] ## 時刻t-1にk=a(t-1)からi = a(t)への遷移発生尤度をkTLに足していき，これがtでのiの存在尤度になる（それはそう！！）
                        #print(f'状態遷移 k = a(t-1) = {k+1} → a = a(t) = {i+1}（全部リンクid）の遷移発生尤度は{kTL}')

                    SLmatrix[t, i] = kTL ### SLmatrixはindex t に入れる（SL[0]を事前に定義済みなので）

            if t == Ti - 1: # この時刻で最終リンクに確定的に滞在
                #print(f'時刻{t}で最後のリンク')
                SLmatrix[t, ad] = 1

            elif t >= Ti: # この時刻以降は確定的に吸収に滞在
                #print(f'時刻{t}で吸収リンク')             
                SLmatrix[t, aabs] = 1
            
        SLlist.append(SLmatrix)
        TLlist.append(TLmatrix)
    
    return(TLlist) 

In [27]:

###########################
####### 経路選択モデル ######
###########################

###### 即時効用行列 ###### # 混雑内生性考えないならT次元は圧縮可能だが一応→時不変だし，経路長も一緒だし経路パラメタしか入れてないので全リンクの効用が等しくなるよ．
def Mset(x): 
    #Probs_T = Probs.T # Probsを読み込む（内生性バージョン）
    inst = np.zeros((T-1, L, L))
    for t in range(T-1):
        inst_t = np.exp(df_link['length']/10 * x[0]) # + Probs_T[:, t] * x[1])    # + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
        inst_t = pd.concat([inst_t]*L, axis=1) 
        inst_t = inst_t.T
        inst_t_numpy = inst_t.values # DataFrameをNumPy配列に変換
        inst[t, :, :] = inst_t_numpy

    return inst


###### 価値関数更新 ###### # 状態価値関数なので状態の数＝Tだけある
"""
# 山野さんのコードは遷移回数がTなので状態数はT+1，今回のコードは状態数がTなので遷移回数はT-1
def newV(x):
    beta = x[-1]
    z = np.ones((T, L, OD)) # なんでODが第三成分なのかがよくわからないが
    V = np.zeros((T, L, OD))

    for od in range(OD):
        # print(od_list.loc[od, 'userid'])
        Id = Ilist[od]

        ##### 即時効用 #####
        M = np.zeros((T-1, L, L))
        for ts in range(T-1):
            Mts = Id[ts, :, :] * Mset(x)[ts, :, :] # 内生性考慮してないのでMsetは定常
            M[ts, :, :] = Mts

        ##### 価値関数 ##### 
        for t in range(T-1, 0, -1): # T-1から1までのT-1回
            zi = np.dot(M[t-1, :, :], (z[t, :, od] ** beta) )
            z[t-1, :, od] = (zi==0)*1 + (zi!=0)*zi
            V[t-1, :, od] = np.log(z[t-1, :, od])

    return z
"""

###### リンク選択確率計算 ######
def newPall(x):
    Pall = np.zeros((OD, T-1, L, L)) # 個人ごと，時刻ごとの各リンク間遷移確率行列 遷移の回数なのでT-1．今回Tを状態の数としているので遷移数はT-1になる
    beta = x[-1]

    #### 以降個人ごとに処理 #### 
    for od in range(OD): 
        
        Id = Ilist[od] # すでに個人のプリズムはIlistで用意してある

        ##### 即時効用 #####
        M = np.zeros((T-1, L, L))
        for ts in range(T-1):
            Mts = Id[ts, :, :] * Mset(x)[ts, :, :] # 内生性考慮してないのでMsetは定常
            M[ts, :, :] = Mts
            
            ##### 価値関数 ##### ### ここで価値関数求解（newVに相当）が入っている
            
        z = np.ones((T, L))
        for t in range(T-1, 0, -1):

                ####
            zii = M[t-1, :, :] * (z[t, :] ** beta) 
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi==0)*1 + (zi!=0)*zi

        ##### 選択確率行列 ##### # 尤度計算は不要
        for t in range(T-1): # 選択確率は各時刻tごとに決まる
            for k in range(L):
                for a in range(L):
                    if M[t, k, a] == 0: # 接続条件を満たせなかった観測は排除（logzero回避）
                        continue # ここがbreakになってたのが癌だった
            
                    Pall[od, t, k, a] += np.exp(np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])) 

    return Pall # 個人ごとの各時刻ごとのリンク遷移確率の一覧がPallに入った


###### リンク同時滞在確率 ######
def Pjoint(Pall): # 各遷移確率を入力として時刻tとt-1のa, kの同時確率
    pjoint = np.zeros((OD, T-1, L, L))
    
    ## pkは状態到達確率
    pk = np.zeros((OD, T, L)) # pk[:, T, :]は使わないからいいや
    # pk[:, 0, :] = np.eye(OD) ## これでも同じ
    # np.fill_diagonal(pk[:, 0, :], 1) ## maybe 全個人odに対してp(a0)=1としているはず
    ### いやこれ普通に意味わからん．各個人のaoに対応するpk[ao]=1としないといけないのでは．

    for od in range(OD):
        #print(f'個人{od}の0列目は{pk[od, 0, :]}') ## ここで全部1になってるとか？
        ao = od_list.loc[od, 'o'] - 1
        pk[od, 0, ao] = 1 # 状態確率なので時刻0では起点Oの確率1，他は0のまま
        ## 全部1になっていたら
        for t in range(T-1): ## p(a0)=1~p(aT-1)=1までのT個　## p(a1, a0)~p(aT-1, aT-2)のT-1個
            pk[od, t+1, :] = pk[od, t, :] @ Pall[od, t, :, :] # 行列積
            #print(f'個人{od}の時刻{t}での状態確率{pk[od, t, :]}') ## 多分解消
    for t in range(T-1):
        for k in range(L):
            for a in range(L):
                ptk = pk[:, t, k]
                ptka = Pall[:, t, k, a]

                pjoint[:, t, k, a] = ptk * ptka ### *は要素ごとの積
    
    return pjoint # pjoint[od, t, k, a]は個人odの時刻tのkとaの同時滞在分布



In [32]:

def KL(x): 
    KL = 0
    LL = 0
    Pt = newPall(x)
    pjoint = Pjoint(Pt)

    TLlist = Mmodel(Pt, b)
    for od in range(OD):
        userid = int(od_list.loc[od, 'userid'])
        TLmatrix = TLlist[od]
        for t in range(T-1):
            KLit = 0
            pit = pjoint[od, t, :, :] # 時刻tにk(a_t-1)とa(a_t)の同時滞在確率はpjoint[t-1]に入っている
            pit = (pit == 0) * 1 + (pit != 0) * pit

            for k in range(L): # 時刻tにa(t-1)からa(tへの遷移を考える)
                for a in range(L):
                    logpka = 0
                    ## 遷移確率ではなく同時滞在確率と同時滞在尤度を比較するという構図にする
                    if pit[k, a] == 0:
                        logpka = 0
                        continue
                    
                    if TLmatrix[t, k, a] == 0:
                        continue

                    else:
                        p = pit[k, a] # 同時滞在確率
                        q = TLmatrix[t, k, a] # 同時滞在尤度
                        #print(f'個人{userid}時刻{t}の{k+1}と{a+1}の同時滞在確率は{p}，同時発生尤度は{q}')
                        print(f'時刻{t}に個人{userid}が{k+1}から{a+1}へ遷移する確率{p}と尤度{q}')

                    qlp = q * np.log(p)
                    qlq = q * np.log(q)

                    # KLit += (qlq - qlp)
                    KLit += qlp # KLではなくなく交差エントロピー（対象事象が揃ったのでどっちもで計算できるはず）
            
            KL += KLit
            #print(f'user{userid}，時刻{t}の一般化平均情報量{-KLit}（正，小さい方がいい）')

    print(f'パラメタ{x}とb{b}での一般化平均情報量{-KL}')

    return -KL




In [10]:
x1 = np.array([-1.5,  0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.36560496268383963と尤度0.028448775495748795
時刻0に個人1006が7から8へ遷移する確率0.5916118070879333と尤度0.9382487781908242
時刻0に個人1006が7から34へ遷移する確率0.04278323022822723と尤度0.03330244631342718
時刻1に個人1006が8から7へ遷移する確率0.11995382556147963と尤度0.00032216374663532766
時刻1に個人1006が8から8へ遷移する確率0.2042035402499263と尤度0.2549990782338656
時刻1に個人1006が8から9へ遷移する確率0.2042035402499263と尤度0.5203859695424251
時刻1に個人1006が8から30へ遷移する確率0.06325090102660133と尤度0.16254156666789818
時刻2に個人1006が8から8へ遷移する確率0.14662728712916265と尤度0.002856007860612708
時刻2に個人1006が8から9へ遷移する確率0.14662728712916265と尤度0.1752511116491237
時刻2に個人1006が8から30へ遷移する確率0.06443205140691863と尤度0.07689195872412916
時刻2に個人1006が9から8へ遷移する確率0.06890825025998742と尤度0.005828359968433606
時刻2に個人1006が9から9へ遷移する確率0.06890825025998742と尤度0.3576413698455687
時刻2に個人1006が9から30へ遷移する確率0.030280175061830453と尤度0.15691623972842275
時刻2に個人1006が30から31へ遷移する確率0.06325090102660133と尤度0.16254156666789818
時刻3に個人1006が9から30へ遷移する確率0.04948443758056604と尤度0.5328924814946924
時刻3に個人1006が30から31へ遷移する確率0.094712226

In [11]:
x1 = np.array([0, 0]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3333333333333333と尤度0.031862683833216845
時刻0に個人1006が7から8へ遷移する確率0.3333333333333333と尤度0.6493996242278322
時刻0に個人1006が7から34へ遷移する確率0.3333333333333333と尤度0.31873769193895096
時刻1に個人1006が8から7へ遷移する確率0.08333333333333333と尤度0.0002738185028521265
時刻1に個人1006が8から8へ遷移する確率0.08333333333333333と尤度0.12731382631497518
時刻1に個人1006が8から9へ遷移する確率0.08333333333333333と尤度0.2598139938463336
時刻1に個人1006が8から30へ遷移する確率0.08333333333333333と尤度0.2619979855636713
時刻2に個人1006が7から8へ遷移する確率0.125と尤度0.0002738185028521265
時刻2に個人1006が8から8へ遷移する確率0.0625と尤度0.00102979420026481
時刻2に個人1006が8から9へ遷移する確率0.0625と尤度0.06319050477946196
時刻2に個人1006が8から30へ遷移する確率0.0625と尤度0.0630935273352484
時刻2に個人1006が9から8へ遷移する確率0.020833333333333332と尤度0.00210153878612256
時刻2に個人1006が9から9へ遷移する確率0.020833333333333332と尤度0.12895518024334737
時刻2に個人1006が9から30へ遷移する確率0.020833333333333332と尤度0.12875727481686366
時刻2に個人1006が30から31へ遷移する確率0.08333333333333333と尤度0.2619979855636713
時刻3に個人1006が8から30へ遷移する確率0.052083333333333336と尤度0.0034051514892394965
時刻3に個人1006が9から30へ遷移

In [12]:
x1 = np.array([-2, 0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.36560496268383896と尤度0.02844877549574874
時刻0に個人1006が7から8へ遷移する確率0.5916118070879333と尤度0.938248778190824
時刻0に個人1006が7から34へ遷移する確率0.042783230228227304と尤度0.033302446313427235
時刻1に個人1006が8から7へ遷移する確率0.11995382556147985と尤度0.0003221637466353277
時刻1に個人1006が8から8へ遷移する確率0.20420354024992665と尤度0.2549990782338656
時刻1に個人1006が8から9へ遷移する確率0.20420354024992665と尤度0.5203859695424252
時刻1に個人1006が8から30へ遷移する確率0.06325090102660133と尤度0.1625415666678979
時刻2に個人1006が8から8へ遷移する確率0.14662728712916248と尤度0.0028560078606127072
時刻2に個人1006が8から9へ遷移する確率0.14662728712916248と尤度0.1752511116491236
時刻2に個人1006が8から30へ遷移する確率0.06443205140691867と尤度0.07689195872412927
時刻2に個人1006が9から8へ遷移する確率0.06890825025998742と尤度0.005828359968433605
時刻2に個人1006が9から9へ遷移する確率0.06890825025998742と尤度0.3576413698455686
時刻2に個人1006が9から30へ遷移する確率0.03028017506183051と尤度0.15691623972842303
時刻2に個人1006が30から31へ遷移する確率0.06325090102660155と尤度0.1625415666678979
時刻3に個人1006が9から30へ遷移する確率0.049484437580566094と尤度0.5328924814946923
時刻3に個人1006が30から31へ遷移する確率0.094712226

In [13]:
x1 = np.array([-5, 0.9]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.35593244195974805と尤度0.02678669848158155
時刻0に個人1006が7から8へ遷移する確率0.6252634505957547と尤度0.9590568173262196
時刻0に個人1006が7から34へ遷移する確率0.01880410744448937と尤度0.014156484192198896
時刻1に個人1006が8から7へ遷移する確率0.11582174517324959と尤度0.00029359540908701417
時刻1に個人1006が8から8へ遷移する確率0.21851121129812168と尤度0.25754057409891745
時刻1に個人1006が8から9へ遷移する確率0.21851121129812168と尤度0.5255724933486411
時刻1に個人1006が8から30へ遷移する確率0.07241928282626116と尤度0.17565015446957397
時刻2に個人1006が8から8へ遷移する確率0.1528643036239219と尤度0.0027813311081037197
時刻2に個人1006が8から9へ遷移する確率0.1528643036239219と尤度0.17066877696019217
時刻2に個人1006が8から30へ遷移する確率0.07543376456322423と尤度0.08409046603062158
時刻2に個人1006が9から8へ遷移する確率0.07404047006511365と尤度0.005675964381258072
時刻2に個人1006が9から9へ遷移する確率0.07404047006511365と尤度0.3482900314157075
時刻2に個人1006が9から30へ遷移する確率0.036536661958588276と尤度0.17160649755167554
時刻2に個人1006が30から31へ遷移する確率0.07241928282626116と尤度0.17565015446957397
時刻3に個人1006が9から30へ遷移する確率0.05904549672205872と尤度0.5189588083758997
時刻3に個人1006が30から31へ遷移する確率0.111970

In [14]:
x1 = np.array([-10, 0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度0.028448775495748552
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度0.9382487781908243
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度0.03330244631342725
時刻1に個人1006が8から7へ遷移する確率0.11995382556147899と尤度0.0003221637466353267
時刻1に個人1006が8から8へ遷移する確率0.20420354024992593と尤度0.25499907823386564
時刻1に個人1006が8から9へ遷移する確率0.20420354024992593と尤度0.5203859695424254
時刻1に個人1006が8から30へ遷移する確率0.06325090102660111と尤度0.16254156666789799
時刻2に個人1006が8から8へ遷移する確率0.14662728712916148と尤度0.0028560078606127072
時刻2に個人1006が8から9へ遷移する確率0.14662728712916148と尤度0.17525111164912366
時刻2に個人1006が8から30へ遷移する確率0.06443205140691823と尤度0.07689195872412927
時刻2に個人1006が9から8へ遷移する確率0.06890825025998695と尤度0.005828359968433607
時刻2に個人1006が9から9へ遷移する確率0.06890825025998695と尤度0.35764136984556877
時刻2に個人1006が9から30へ遷移する確率0.030280175061830294と尤度0.15691623972842306
時刻2に個人1006が30から31へ遷移する確率0.06325090102660111と尤度0.16254156666789799
時刻3に個人1006が9から30へ遷移する確率0.049484437580565754と尤度0.5328924814946925
時刻3に個人1006が30から31へ遷移する確率0.0947

In [15]:
x1 = np.array([-20, 0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf) 
print(KL(x1)) 

時刻0に個人1006が7から7へ遷移する確率0.36560496268384024と尤度0.028448775495749694
時刻0に個人1006が7から8へ遷移する確率0.5916118070879144と尤度0.9382487781908222
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度0.033302446313428116
時刻1に個人1006が8から7へ遷移する確率0.1199538255614773と尤度0.00032216374663532517
時刻1に個人1006が8から8へ遷移する確率0.20420354024992304と尤度0.2549990782338645
時刻1に個人1006が8から9へ遷移する確率0.20420354024992304と尤度0.5203859695424229
時刻1に個人1006が8から30へ遷移する確率0.06325090102660111と尤度0.16254156666789957
時刻2に個人1006が8から8へ遷移する確率0.14662728712916256と尤度0.002856007860612707
時刻2に個人1006が8から9へ遷移する確率0.14662728712916256と尤度0.17525111164912358
時刻2に個人1006が8から30へ遷移する確率0.0644320514069178と尤度0.07689195872412817
時刻2に個人1006が9から8へ遷移する確率0.06890825025998695と尤度0.005828359968433603
時刻2に個人1006が9から9へ遷移する確率0.06890825025998695と尤度0.3576413698455685
時刻2に個人1006が9から30へ遷移する確率0.030280175061829867と尤度0.15691623972842075
時刻2に個人1006が30から31へ遷移する確率0.06325090102660111と尤度0.16254156666789957
時刻3に個人1006が9から30へ遷移する確率0.049484437580564596と尤度0.532892481494692
時刻3に個人1006が30から31へ遷移する確率0.0947122

In [16]:
x1 = np.array([1, 0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.36560496268383896と尤度0.02844877549574874
時刻0に個人1006が7から8へ遷移する確率0.5916118070879333と尤度0.938248778190824
時刻0に個人1006が7から34へ遷移する確率0.042783230228227304と尤度0.033302446313427235
時刻1に個人1006が8から7へ遷移する確率0.11995382556147963と尤度0.0003221637466353276
時刻1に個人1006が8から8へ遷移する確率0.2042035402499263と尤度0.25499907823386553
時刻1に個人1006が8から9へ遷移する確率0.2042035402499263と尤度0.520385969542425
時刻1に個人1006が8から30へ遷移する確率0.06325090102660133と尤度0.16254156666789815
時刻2に個人1006が8から8へ遷移する確率0.14662728712916265と尤度0.002856007860612711
時刻2に個人1006が8から9へ遷移する確率0.14662728712916237と尤度0.17525111164912355
時刻2に個人1006が8から30へ遷移する確率0.06443205140691863と尤度0.07689195872412924
時刻2に個人1006が9から8へ遷移する確率0.06890825025998742と尤度0.005828359968433612
時刻2に個人1006が9から9へ遷移する確率0.0689082502599873と尤度0.3576413698455684
時刻2に個人1006が9から30へ遷移する確率0.030280175061830453と尤度0.15691623972842292
時刻2に個人1006が30から31へ遷移する確率0.06325090102660133と尤度0.16254156666789815
時刻3に個人1006が9から30へ遷移する確率0.04948443758056594と尤度0.5328924814946919
時刻3に個人1006が30から31へ遷移する確率0.0947122264

In [17]:
x1 = np.array([-10, 0]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3333333333333329と尤度0.031862683833216845
時刻0に個人1006が7から8へ遷移する確率0.3333333333333329と尤度0.6493996242278322
時刻0に個人1006が7から34へ遷移する確率0.3333333333333329と尤度0.31873769193895096
時刻1に個人1006が8から7へ遷移する確率0.08333333333333322と尤度0.0002738185028521265
時刻1に個人1006が8から8へ遷移する確率0.08333333333333322と尤度0.12731382631497518
時刻1に個人1006が8から9へ遷移する確率0.08333333333333322と尤度0.2598139938463336
時刻1に個人1006が8から30へ遷移する確率0.08333333333333322と尤度0.2619979855636713
時刻2に個人1006が7から8へ遷移する確率0.12500000000000022と尤度0.0002738185028521265
時刻2に個人1006が8から8へ遷移する確率0.062499999999999986と尤度0.00102979420026481
時刻2に個人1006が8から9へ遷移する確率0.062499999999999986と尤度0.06319050477946196
時刻2に個人1006が8から30へ遷移する確率0.062499999999999986と尤度0.0630935273352484
時刻2に個人1006が9から8へ遷移する確率0.0208333333333333と尤度0.00210153878612256
時刻2に個人1006が9から9へ遷移する確率0.0208333333333333と尤度0.12895518024334737
時刻2に個人1006が9から30へ遷移する確率0.0208333333333333と尤度0.12875727481686366
時刻2に個人1006が30から31へ遷移する確率0.08333333333333322と尤度0.2619979855636713
時刻3に個人1006が8から30へ遷移する確率0.052083333333

こちらの尤度比は0.4~を示している．まあこっちの方が，初期のOを与えた時にどのあたりに行き着いているかのモデルによる予測分布と，観測をもとに（初期のOを確定的に与えた際に）どこにいそうかの観測尤度を比較しているので，比較事象の比較としてはこちらの方が適切にも思えるか．ただしPの正体が単純なp(a|k)ではない点が美しくない気もする

In [30]:
x1 = np.array([-10, 0.8]) 
b = np.array([4, 20]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度0.06874181101392843
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度0.8934710964200657
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度0.0377870925660058
時刻1に個人1006が7から7へ遷移する確率0.1352914310875274と尤度0.0006135696672925342
時刻1に個人1006が7から8へ遷移する確率0.23031353159631024と尤度0.06812824134663589
時刻1に個人1006が8から7へ遷移する確率0.11995382556147899と尤度0.0019270917790363735
時刻1に個人1006が8から8へ遷移する確率0.20420354024992593と尤度0.21397631078902835
時刻1に個人1006が8から9へ遷移する確率0.20420354024992593と尤度0.5175469351104965
時刻1に個人1006が8から30へ遷移する確率0.06325090102660111と尤度0.1600207587415045
時刻2に個人1006が8から8へ遷移する確率0.14662728712916148と尤度0.010661030616767844
時刻2に個人1006が8から9へ遷移する確率0.14662728712916148と尤度0.18906077490951448
時刻2に個人1006が8から30へ遷移する確率0.06443205140691823と尤度0.08238274660938188
時刻2に個人1006が9から8へ遷移する確率0.06890825025998695と尤度0.019558648306298707
時刻2に個人1006が9から9へ遷移する確率0.06890825025998695と尤度0.34684950619648147
時刻2に個人1006が9から30へ遷移する確率0.030280175061830294と尤度0.1511387806077164
時刻2に個人1006が30から31へ遷移する確率0.06325090102

In [31]:
x1 = np.array([-10, 0.8]) 
b = np.array([1.5, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度0.08057747416913842
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度0.8735988702118651
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度0.0458236556189966
時刻1に個人1006が7から7へ遷移する確率0.1352914310875274と尤度0.0007691979616606043
時刻1に個人1006が7から8へ遷移する確率0.23031353159631024と尤度0.07980827620747782
時刻1に個人1006が8から7へ遷移する確率0.11995382556147899と尤度0.004941891301936257
時刻1に個人1006が8から8へ遷移する確率0.20420354024992593と尤度0.5127468423873501
時刻1に個人1006が8から9へ遷移する確率0.20420354024992593と尤度0.27001671900896795
時刻1に個人1006が8から30へ遷移する確率0.06325090102660111と尤度0.08589341751361085
時刻2に個人1006が8から8へ遷移する確率0.14662728712916148と尤度0.021378858060333572
時刻2に個人1006が8から9へ遷移する確率0.14662728712916148と尤度0.3950290419191613
時刻2に個人1006が8から30へ遷移する確率0.06443205140691823と尤度0.17614721861533306
時刻2に個人1006が9から8へ遷移する確率0.06890825025998695と尤度0.009741961428497712
時刻2に個人1006が9から9へ遷移する確率0.06890825025998695と尤度0.18000763551787335
時刻2に個人1006が9から30へ遷移する確率0.030280175061830294と尤度0.08026712206259691
時刻2に個人1006が30から31へ遷移する確率0.06325090102

In [33]:
x1 = np.array([-10, 0.8]) 
b = np.array([2.5, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度0.019958960248376616
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度0.9503885458660801
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度0.02965249388554321
時刻1に個人1006が8から7へ遷移する確率0.11995382556147899と尤度0.00011665390510421057
時刻1に個人1006が8から8へ遷移する確率0.20420354024992593と尤度0.1823890668315648
時刻1に個人1006が8から9へ遷移する確率0.20420354024992593と尤度0.585971882909364
時刻1に個人1006が8から30へ遷移する確率0.06325090102660111と尤度0.18191094222004708
時刻2に個人1006が8から8へ遷移する確率0.14662728712916148と尤度0.001264762280725304
時刻2に個人1006が8から9へ遷移する確率0.14662728712916148と尤度0.12614082377318633
時刻2に個人1006が8から30へ遷移する確率0.06443205140691823と尤度0.05498348077765316
時刻2に個人1006が9から8へ遷移する確率0.06890825025998695と尤度0.0040633747841572285
時刻2に個人1006が9から9へ遷移する確率0.06890825025998695と尤度0.40525990566294356
時刻2に個人1006が9から30へ遷移する確率0.030280175061830294と尤度0.1766486024622632
時刻2に個人1006が30から31へ遷移する確率0.06325090102660111と尤度0.18191094222004708
時刻3に個人1006が9から30へ遷移する確率0.049484437580565754と尤度0.5314007294361299
時刻3に個人1006が30から31へ遷移する確率0.094712

In [34]:
x1 = np.array([-10, 0.8]) 
b = np.array([2.5, 20]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度0.08108492178553282
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度0.8792862376065634
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度0.03962884060790372
時刻1に個人1006が7から7へ遷移する確率0.1352914310875274と尤度0.0010132437670062966
時刻1に個人1006が7から8へ遷移する確率0.23031353159631024と尤度0.08007167801852652
時刻1に個人1006が8から7へ遷移する確率0.11995382556147899と尤度0.00314431114533843
時刻1に個人1006が8から8へ遷移する確率0.20420354024992593と尤度0.24847946547302915
時刻1に個人1006が8から9へ遷移する確率0.20420354024992593と尤度0.4790814614490511
時刻1に個人1006が8から30へ遷移する確率0.06325090102660111と尤度0.14858099953914466
時刻2に個人1006が8から8へ遷移する確率0.14662728712916148と尤度0.016313461615934947
時刻2に個人1006が8から9へ遷移する確率0.14662728712916148と尤度0.21721880827093562
時刻2に個人1006が8から30へ遷移する確率0.06443205140691823と尤度0.09501887360468513
時刻2に個人1006が9から8へ遷移する確率0.06890825025998695と尤度0.02378770303216426
時刻2に個人1006が9から9へ遷移する確率0.06890825025998695と尤度0.31674065417865677
時刻2に個人1006が9から30へ遷移する確率0.030280175061830294と尤度0.1385531042382301
時刻2に個人1006が30から31へ遷移する確率0.063250901026

In [35]:
x1 = np.array([-10, 0.8]) 
b = np.array([4, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度0.014590185707479164
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度0.9586956312332869
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度0.026714183059233832
時刻1に個人1006が8から7へ遷移する確率0.11995382556147899と尤度4.5934608872779646e-05
時刻1に個人1006が8から8へ遷移する確率0.20420354024992593と尤度0.1314662610512701
時刻1に個人1006が8から9へ遷移する確率0.20420354024992593と尤度0.6320355457554551
時刻1に個人1006が8から30へ遷移する確率0.06325090102660111と尤度0.1951478898176891
時刻2に個人1006が8から8へ遷移する確率0.14662728712916148と尤度0.000550403916476777
時刻2に個人1006が8から9へ遷移する確率0.14662728712916148と尤度0.09136384998522581
時刻2に個人1006が8から30へ遷移する確率0.06443205140691823と尤度0.03955200714956749
時刻2に個人1006が9から8へ遷移する確率0.06890825025998695と尤度0.002646114957210756
時刻2に個人1006が9から9へ遷移する確率0.06890825025998695と尤度0.4392396978964197
時刻2に個人1006が9から30へ遷移する確率0.030280175061830294と尤度0.19014973290182458
時刻2に個人1006が30から31へ遷移する確率0.06325090102660111と尤度0.1951478898176891
時刻3に個人1006が9から30へ遷移する確率0.049484437580565754と尤度0.5306035478816455
時刻3に個人1006が30から31へ遷移する確率0.0947122

In [36]:
x1 = np.array([-10, 0.8]) 
b = np.array([4, 5]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度2.0587934006014144e-15
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度0.9999864590057758
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度1.3540994222042631e-05
時刻1に個人1006が8から7へ遷移する確率0.11995382556147899と尤度3.066866827981097e-36
時刻1に個人1006が8から8へ遷移する確率0.20420354024992593と尤度5.602349581831579e-07
時刻1に個人1006が8から9へ遷移する確率0.20420354024992593と尤度0.7686257575664591
時刻1に個人1006が8から30へ遷移する確率0.06325090102660111と尤度0.2313601412043585
時刻2に個人1006が9から8へ遷移する確率0.06890825025998695と尤度5.803767580548512e-21
時刻2に個人1006が9から9へ遷移する確率0.06890825025998695と尤度0.5553233065386382
時刻2に個人1006が9から30へ遷移する確率0.030280175061830294と尤度0.21330245102782103
時刻2に個人1006が30から31へ遷移する確率0.06325090102660111と尤度0.2313601412043585
時刻3に個人1006が9から30へ遷移する確率0.049484437580565754と尤度0.5553233065386382
時刻3に個人1006が30から31へ遷移する確率0.09471222646874852と尤度0.21330245102782103
時刻3に個人1006が31から24へ遷移する確率0.06325090102660111と尤度0.2313601412043585
時刻4に個人1006が24から24へ遷移する確率0.03162545051330049と尤度1.9123584837527976e-07
時刻4に個人1006が24から25へ遷移

In [37]:
x1 = np.array([-10, 0.8]) 
b = np.array([1, 5]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度0.999999999999815
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度8.624351942174168e-21
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度1.8499005357063632e-13
時刻1に個人1006が7から7へ遷移する確率0.1352914310875274と尤度0.999999999999815
時刻1に個人1006が7から8へ遷移する確率0.23031353159631024と尤度1.290597149979501e-33
時刻2に個人1006が7から8へ遷移する確率0.16748545230736744と尤度0.999999999999815
時刻3に個人1006が8から30へ遷移する確率0.08793713782086506と尤度0.999999999999815
時刻7に個人1006が25から46へ遷移する確率1.0と尤度1.0
時刻8に個人1006が46から46へ遷移する確率1.0と尤度1.0
時刻9に個人1006が46から46へ遷移する確率1.0と尤度1.0
時刻4に個人141が17から47へ遷移する確率1.0と尤度1.0
時刻5に個人141が47から47へ遷移する確率1.0と尤度1.0
時刻6に個人141が47から47へ遷移する確率1.0と尤度1.0
時刻7に個人141が47から47へ遷移する確率1.0と尤度1.0
時刻8に個人141が47から47へ遷移する確率1.0と尤度1.0
時刻9に個人141が47から47へ遷移する確率1.0と尤度1.0
時刻4に個人424が25から46へ遷移する確率1.0と尤度1.0
時刻5に個人424が46から46へ遷移する確率1.0と尤度1.0
時刻6に個人424が46から46へ遷移する確率1.0と尤度1.0
時刻7に個人424が46から46へ遷移する確率1.0と尤度1.0
時刻8に個人424が46から46へ遷移する確率1.0と尤度1.0
時刻9に個人424が46から46へ遷移する確率1.0と尤度1.0
時刻7に個人626が18から48へ遷移する確率1.0と尤度1.0
時刻8に個人626が48から48へ遷移する確率

In [38]:
x1 = np.array([-10, 0.8]) 
b = np.array([2, 5]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度1.0182542362691218e-12
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度0.9998804698835629
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度0.00011953011541884947
時刻1に個人1006が8から7へ遷移する確率0.11995382556147899と尤度7.053772293941468e-28
時刻1に個人1006が8から8へ遷移する確率0.20420354024992593と尤度0.001219510692265383
時刻1に個人1006が8から9へ遷移する確率0.20420354024992593と尤度0.7486318953436237
時刻1に個人1006が8から30へ遷移する確率0.06325090102660111と尤度0.25002906384767376
時刻2に個人1006が9から8へ遷移する確率0.06890825025998695と尤度4.234381266321757e-17
時刻2に個人1006が9から9へ遷移する確率0.06890825025998695と尤度0.5222788112155057
時刻2に個人1006が9から30へ遷移する確率0.030280175061830294と尤度0.226353084128118
時刻2に個人1006が30から31へ遷移する確率0.06325090102660111と尤度0.25002906384767376
時刻3に個人1006が9から30へ遷移する確率0.049484437580565754と尤度0.5222788112155057
時刻3に個人1006が30から31へ遷移する確率0.09471222646874852と尤度0.226353084128118
時刻3に個人1006が31から24へ遷移する確率0.06325090102660111と尤度0.25002906384767376
時刻4に個人1006が24から24へ遷移する確率0.03162545051330049と尤度0.010480199446919325
時刻4に個人1006が24から25へ遷移する確率

In [40]:
x1 = np.array([-10, 0.8]) 
b = np.array([1.5, 5]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度2.2624051920858434e-08
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度0.9959971835776444
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度0.004002793798303766
時刻1に個人1006が8から7へ遷移する確率0.11995382556147899と尤度5.019751295478389e-17
時刻1に個人1006が8から8へ遷移する確率0.20420354024992593と尤度0.991692753796888
時刻1に個人1006が8から9へ遷移する確率0.20420354024992593と尤度0.003088631463296172
時刻1に個人1006が8から30へ遷移する確率0.06325090102660111と尤度0.0012157983174602702
時刻2に個人1006が8から8へ遷移する確率0.14662728712916148と尤度2.630875595094642e-12
時刻2に個人1006が8から9へ遷移する確率0.14662728712916148と尤度0.6605428657842524
時刻2に個人1006が8から30へ遷移する確率0.06443205140691823と尤度0.3311498880100048
時刻3に個人1006が9から30へ遷移する確率0.049484437580565754と尤度0.6605428657842524
時刻3に個人1006が30から31へ遷移する確率0.09471222646874852と尤度0.3311498880100048
時刻4に個人1006が30から31へ遷移する確率0.13742157540142885と尤度0.6605428657842524
時刻4に個人1006が31から24へ遷移する確率0.09471222646874852と尤度0.3311498880100048
時刻5に個人1006が24から24へ遷移する確率0.0631688384910253と尤度3.305870637341081e-21
時刻5に個人1006が24から25へ遷移する確率0.